In [1]:
import re

original_line_lst: list = [
        "20 Q. Now, you referenced reviewing some",
        ]           

detect_pages = False

for i, e in enumerate(original_line_lst):
    line = e.strip()
    
line

'20 Q. Now, you referenced reviewing some'

Separate the line number from the other text and track line numbers for later

In [2]:
original_line = '20 Q. Now, you referenced reviewing some'

def set_pattern_match(text: str) -> str:
    first_num = None  # set up variables to keep track of line number range
    last_num = None  # set up variables to keep track of line number range
    if not detect_pages:
        match = re.match(r'^\d+(:\s*\d+)?\s+', text)

        if match:
            num = match.group(1)
            text = text[match.end():].lstrip()
    else:
        match = re.match(r'^(\d+)\s+', text)

        if match:
            num = int(match.group(1))

            if first_num is None:
                first_num = num
            last_num = num  # Always update last_num with the latest number

            text = text[match.end():].lstrip()
    return text

# text_line: str = set_pattern_match(line)
# text_line

Replace any words in the word dictionary

In [3]:
text_line = 'THE WITNESS: Now, you referenced reviewing some'

def dict_word_replacement(text: str) -> str:
    """ does a key/value word replacement for any of the text strings in the current line """
    swap_phrase_dict = {"THE WITNESS:": "A."}
    for word, replacement in swap_phrase_dict.items():
        if word in text:
            text = text.replace(word, replacement)
    return text
    
text_line: str = dict_word_replacement(text_line)
text_line: str
text_line

'A. Now, you referenced reviewing some'

Functions to filter and/or reformat one line at a time

In [4]:
def qa_phrase_filter(text: str) -> list:
    """ checks for lines that start with Q. or A. and also checks if the current line is caps or if hide_obj """
    qa_phrases = ["Q.", "A."]
    if any(text.startswith(phrase) for phrase in qa_phrases):
        return [f'<QA>: {text}', False]
    else:
        return [f'{text}', False]
    

Check for Objection Phrases at the beginning of a line

In [5]:
# def objection_phrases_filter(lst: list) -> str:
#     """ checks for lines that start with one of the phrases the generally start an objection and also checks if the current line is caps or if hide_obj"""
#     output_line = []
#     objection_phrases = ["MR", "MRS", "MS", "ATTY", "ATTORNEY"]
#     if any(lst[0].startswith(phrase) for phrase in objection_phrases):
#         output_line = [f'<OBJ>: {lst[0]}', True]
#     else:
#         output_line = [{lst[0]}, False]
#     return output_line
        # filtered_line_list.append(text.upper())
        # is_cap = True
        # return ["".join(filtered_line_list), is_cap]

# lines_lst: list = objection_phrases_filter(text_line)
# lines_lst

In [6]:
# def name_lines_only_filter(text: str):
#     if text.startswith("QUESTIONS BY") or text.startswith("BY") and ":" in text:
#         return f'<NAMES>: {text.upper()}'
#     else:
#         return text
    
# def check_for_pg_text(text: str) -> str:    
#     if text.startswith("Pg. "):
#         text = "\n\n" + text
#     return text

In [7]:
# def handling_other_lines(text: str):
#     if text and not any(line.startswith(phrase) for phrase in qa_phrases):
#         phrase_being_assembled += " "
#         phrase_being_assembled += line

Now let's loop through some lines

In [8]:
original_line_lst: list = [
"8 A. There were a lot of documents. I don't",
"9 remember any specific document. There were lots.",
"10 BY MR. HOLLANDER:",
"11 Q. Can you describe the categories of",
"12 documents that you reviewed?",
"13 MR. DORSKY: Same caution, came",
"14 objection.",
"15 A. No, I really don't remember.",
"16 BY MR. HOLLANDER:",
"17 Q. Do you have any documents or notes with",
"18 you today?",
"19 A. No, I do not."
]

In [9]:
def tag_all_lines(text: str, line_lst) -> list:
    qa_prefixes = ["Q.", "A."]
    objection_phrases = ["MR", "MRS", "MS", "ATTY", "ATTORNEY"]
    name_phrases = []
    if any(text.startswith(prefix) for prefix in qa_prefixes):
        line_lst.append([f'<QA> {text}', "QA", True])
    elif any(text.startswith(phrase) for phrase in objection_phrases):
        line_lst.append([f'<OBJECTION> {text.upper()}', "OBJECTION", True])
    else:
        line_lst.append([text, "tween", False])
    return line_lst

In [13]:

line_lst = []
for i, e in enumerate(original_line_lst):
    text_line = e.strip()
    text_line:str = set_pattern_match(text_line)
    text_line: str = dict_word_replacement(text_line)
    the_lst = tag_all_lines(text_line, line_lst)
    
# print(i+1, len(the_lst))
# for i, e in enumerate(the_lst):
#     # if i < len(the_lst)-1 and e[1]:
#     line_lst.append(e)
        
the_lst

[["<QA> A. There were a lot of documents. I don't", 'QA', True],
 ['remember any specific document. There were lots.', 'tween', False],
 ['BY MR. HOLLANDER:', 'tween', False],
 ['<QA> Q. Can you describe the categories of', 'QA', True],
 ['documents that you reviewed?', 'tween', False],
 ['<OBJECTION> MR. DORSKY: SAME CAUTION, CAME', 'OBJECTION', True],
 ['objection.', 'tween', False],
 ["<QA> A. No, I really don't remember.", 'QA', True],
 ['BY MR. HOLLANDER:', 'tween', False],
 ['<QA> Q. Do you have any documents or notes with', 'QA', True],
 ['you today?', 'tween', False],
 ['<QA> A. No, I do not.', 'QA', True]]